Reference

- https://github.com/adapter-hub/adapters
- https://docs.adapterhub.ml/loading.html
- https://docs.adapterhub.ml/adapter_composition.html

In [1]:
!pip install -q -U transformers

ERROR: adapter-transformers 3.2.1 has requirement tokenizers!=0.11.3,<0.14,>=0.11.1, but you'll have tokenizers 0.15.0 which is incompatible.
ERROR: refractml 1.0.3 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: refractml 1.0.3 has requirement urllib3==1.26.15, but you'll have urllib3 2.1.0 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: google-auth 2.22.0 has requirement urllib3<2.0, but you'll have urllib3 2.1.0 which is incompatible.


In [2]:
!pip install -U adapters

     |████████████████████████████████| 235kB 7.1MB/s eta 0:00:01
     |████████████████████████████████| 7.9MB 20.5MB/s eta 0:00:01
     |████████████████████████████████| 737kB 114.5MB/s eta 0:00:01
     |████████████████████████████████| 778kB 117.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 102.9MB/s ta 0:00:01
     |████████████████████████████████| 1.3MB 110.3MB/s eta 0:00:01
     |████████████████████████████████| 317kB 117.2MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 105.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 87.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 97.6MB/s eta 0:00:01
     |████████████████████████████████| 3.9MB 115.7MB/s eta 0:00:01
     |████████████████████████████████| 174kB 109.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 79.8MB/s eta 0:00:01
     |████████████████████████████████| 163kB 109.9MB/s eta 0:00:01
     |████████████████████████████████| 143kB 112.6MB/s et

In [1]:
import transformers

In [4]:
from transformers import AutoModelWithLMHead

In [5]:
from transformers import list_adapters

ImportError: cannot import name 'list_adapters' from 'transformers' (/tmp/pip_packages/transformers/__init__.py)